In [6]:
import requests
import json
from pathlib import Path 
import os

In [7]:
BASE_URL = "https://api.eka.care"
CLIENT_ID = <your client id>
CLIENT_SECRET = <your client secret>

# Needed if you are trying lab report parsing
lab_report_path = <file path>

# Needed if you are trying Medical Scribe
audio_file_path = ""

# General functions wrapping APIs

In [8]:
def get_client_token(client_id, client_secret):
    """
    Retrieve JWT token using client credentials.
    """
    url = f"{BASE_URL}/connect-auth/v1/account/login"
    payload = {
        "client_id": client_id,
        "client_secret": client_secret,
    }
    headers = {"Content-Type": "application/json"}
    
    response = requests.post(url, json=payload, headers=headers)
    
    response.raise_for_status()
    return response.json().get("access_token")


def get_mime_type(filepath):
    filename, ext = os.path.splitext(filepath)
    ext = ext.split('.')[-1].lower()
    if ext == 'pdf':
        mime = "application/pdf"
    elif ext in ['jpeg', 'jpg']:
        mime = 'image/jpeg'
    elif ext in ['png']:
        mime = 'image/png'
    else:
        raise Exception("Please provide either PDF, PNG, JPEG, or JPG formatted files")
    return mime
    
    
def upload_lab_report(access_token, file_paths, tasks=None):
    """
    Upload files to the server.
    """
    url = f"{BASE_URL}/mr/api/v2/docs?dt=lr"
    if type(tasks) is list:
        last_task_index = len(tasks) - 1
        for index, task in enumerate(tasks):
            if index <= last_task_index:
                url += "&"
            url += f"task={task}"
    headers = {
        "auth": access_token,
    }

     # Prepare files for upload
    
    files = [
        ("file", (Path(file_path).name, open(file_path, "rb"), get_mime_type(file_path)))
        for file_path in file_paths
    ]

    response = requests.post(url, headers=headers, files=files)

    response.raise_for_status()
    return response.json().get("document_id")

def upload_audio(access_token, file_paths, mode, txnid):
    """
    Upload files to the server.
    """
    url = f"{BASE_URL}/voice/upload"
    headers = {
        "auth": access_token,
    }

    querystring = {"mode":f"{mode}","txnid":f"{txnid}"}

     # Prepare files for upload
    files = [
        ("file", (Path(file_path).name, open(file_path, "rb"), "audio/wav"))
        for file_path in file_paths
    ]

    response = requests.post(url, headers=headers, files=files, params=querystring)

    return response.text


def retrieve_json_data(access_token, document_id):
    """
    Retrieve JSON data using the document ID.
    """
    identifier = f"ekacare-sr-{document_id}"
    url = f"{BASE_URL}/health/api/v1/fhir/retrieve?identifier={identifier}"
    headers = {"auth": access_token}
    
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()

def retrieve_mr_result(access_token, document_id):
    """
    Retrieve JSON data using the document ID.
    """
    # identifier = f"ekacare-sr-{document_id}"
    identifier = f"{document_id}"
    url = f"{BASE_URL}/mr/api/v1/docs/{identifier}/result"
    headers = {"auth": access_token}
    
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()

def retrieve_scribe_output(access_token, txid):
    """
    Retrieve JSON data using the document ID.
    """
    identifier = str(txid)
    url = f"{BASE_URL}/health/api/v1/fhir/retrieve?identifier={identifier}"
    headers = {"auth": access_token}
    
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()

def delete_document(access_token, document_id):
    """
    Delete a document by its ID.
    """
    url = f"{BASE_URL}/mr/api/v1/docs/{document_id}"
    headers = {
        "auth": access_token,
        
    }
    
    response = requests.delete(url, headers=headers)
    response.raise_for_status()
    return response.status_code

### Authenticate

In [11]:
# Step 1: Get client token
access_token = get_client_token(CLIENT_ID, CLIENT_SECRET)
print(f"Access Token: {access_token}")

NameError: name 'CLIENT_ID' is not defined

### Upload file

In [ ]:
# Step 2: Upload files and retrieve document ID

file_paths = [lab_report_path]
document_id = upload_lab_report(access_token, file_paths, tasks=['pii', 'smart'])
print(f"Document ID: {document_id}")

### Result API

In [ ]:
result = retrieve_mr_result(access_token, document_id)

In [ ]:
result

### Fetch smart record

Do this once the status in the above step is completed

NOTE: to avoid polling there is also a webhook method to get notified about completion of report parsing

In [ ]:
# Step 3: Retrieve JSON data
json_data = retrieve_json_data(access_token, document_id)
print("Retrieved JSON Data:")
print(json.dumps(json_data, indent=4))

### DELETE RECORD

In [ ]:
delete_status = delete_document(access_token, document_id)
print(f"Document deleted successfully. Status code: {delete_status}")

# Eka Speech to Medical Record

In [10]:
# Step 2: Upload files and retrieve document ID

file_paths = [audio_file_path]

# According to your workflow, give any meaningful unique transaction id
tx_id = 'niharika-sample-testing'

response = upload_audio(access_token, file_paths, mode='dictation', txnid=tx_id)
print(response)

NameError: name 'access_token' is not defined

### Fetch Structured medical document

NOTE: the response below is available within 2 mins of upload of audio. Status API for this will soon be released

In [ ]:
# Step 3: Retrieve JSON data
json_data = retrieve_scribe_output(access_token, tx_id)
print("Retrieved JSON Data:")
print(json.dumps(json_data, indent=4))